# Submit

## Registration

Let's start with registering your team. Please provide the team name and then First name, Last name, and email addresses for each member of your team.

In [1]:
team_name = input("Enter your team name: ")
print("\n")
data = []

print("For each team memeber please provide:")
while firstname := input("First name (or emply string when done): "):
    lastname = input("Last name: ")
    email = input("Email address: ")
    data.append((firstname, lastname, email))
    print("\n")

Enter your team name:  TestTeamBook




For each team memeber please provide:


First name (or emply string when done):  Vadim
Last name:  Karpusenko
Email address:  karpusenko@ionq.co


First name (or emply string when done):  


Run the code cell below to make sure the information your provided is correct. If you need to change anything - run the previous code cell.

In [2]:
print(f"Team name: {team_name}")
for st in data:
    print(f"{st[0]}\t{st[1]}\t{st[2]}")

Team name: TestTeamBook
Vadim	Karpusenko	karpusenko@ionq.co


In [3]:
import requests
for firstname, lastname, email in data:
    url = f"https://fliq2025.azurewebsites.net/api/registration?TeamName={team_name}&FirstName={firstname}&LastName={lastname}&Email={email}"
    req = requests.post(url)
    key = req.text

`key` will be used to submit your code for evaluation.

Check your email now. You should get registration confirmation link to click. Thank you!

## Code modification

This code cell allows you to modify the code below.  Running the cell saves it to `submit.py` in your local directory, creating a backup for your reference.  **Do not rename `submit.py`**.  Your submission will be evaluated based on the contents of this file, specifically your implementations of `build_ansatz`, `build_maxcut_hamiltonian`, the `QITEvolver` class, and the values of `num_steps` and `lr` (learning rate).


In [4]:
%%writefile submit.py

import networkx as nx
from networkx.generators.degree_seq import random_degree_sequence_graph
import numpy as np
import pandas as pd
import time

from typing import List
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

import random


def ring_mesh_8():
    G = nx.Graph()
    G.add_edges_from([(i, (i + 1) % 8) for i in range(8)])
    return G

def bi_complete_8x8():
    left = list(range(8))
    right = list(range(8, 16))
    G = nx.Graph()
    G.add_nodes_from(left + right)

    for u in left:
        for v in right:
            G.add_edge(u, v)
    return G

def bi_complete_nxn(n):
    left = list(range(n))
    right = list(range(n, 2 * n))
    G = nx.Graph()
    G.add_nodes_from(left + right)
    for u in left:
        for v in right:
            G.add_edge(u, v)
    return G

def reg_graph_8():
    seq = [4] * 8
    tries = 0
    while True:
        G = random_degree_sequence_graph(seq, seed=random.randint(0, 1000))

        if nx.is_connected(G) and all(d == 4 for _, d in G.degree()):
            break
        tries += 1
        if tries > 10:
            break
    return G

def cubic_graph_16():
    G = nx.random_regular_graph(3, 16, seed=1234)
    return G

def random_connected_graph_16(p=0.18):
    n = 16
    tries = 0
    while True:
        G = nx.erdos_renyi_graph(n, p, seed=random.randint(0, 10000))
        if nx.is_connected(G):
            break
        tries += 1
        if tries > 20:
            break
    return G

def expander_graph_n(n):
    G = nx.random_regular_graph(4, n, seed=99)
    return G

def defective_grid_4x4():
    G = nx.Graph()
    G.add_nodes_from(range(16))

    for i in range(16):
        if i % 4 != 3 and not (i == 5):
            G.add_edge(i, i + 1)

    for i in range(12):
        G.add_edge(i, i + 4)

    return G

graph1 = ring_mesh_8()
graph2 = bi_complete_8x8()
graph3 = bi_complete_nxn(5)
graph4 = reg_graph_8()
graph5 = cubic_graph_16()
graph6 = random_connected_graph_16(p=0.18)
graph7 = expander_graph_n(16)
graph8 = defective_grid_4x4()

graph = graph1

#####################################################
# You can edit the code below this line!       #
#####################################################  

num_steps=100 #number of QITE steps
lr=0.1 #learning rate

def build_ansatz(graph: nx.Graph) -> QuantumCircuit:
    
    ansatz = QuantumCircuit(graph.number_of_nodes())
    ansatz.h(range(graph.number_of_nodes()))

    theta = ParameterVector(r"$\theta$", graph.number_of_edges())
    for t, (u, v) in zip(theta, graph.edges):
        #ansatz.cx(u, v)
        ansatz.ry(t, v)
        ansatz.cx(u, v)

    return ansatz

def build_maxcut_hamiltonian(graph: nx.Graph) -> SparsePauliOp:
    """
    Build the MaxCut Hamiltonian for the given graph H = (|E|/2)*I - (1/2)*Σ_{(i,j)∈E}(Z_i Z_j)
    """
    num_qubits = len(graph.nodes)
    edges = list(graph.edges())
    num_edges = len(edges)

    pauli_terms = ["I"*num_qubits] # start with identity
    coeffs = [-num_edges / 2]

    for (u, v) in edges: # for each edge, add -(1/2)*Z_i Z_j
        z_term = ["I"] * num_qubits
        z_term[u] = "Z"
        z_term[v] = "Z"
        pauli_terms.append("".join(z_term))
        coeffs.append(0.5)

    return SparsePauliOp.from_list(list(zip(pauli_terms, coeffs)))

class QITEvolver:
    """
    A class to evolve a parametrized quantum state under the action of an Ising
    Hamiltonian according to the variational Quantum Imaginary Time Evolution
    (QITE) principle described in IonQ's latest joint paper with ORNL.
    """
    def __init__(self, hamiltonian: SparsePauliOp, ansatz: QuantumCircuit):
        self.hamiltonian = hamiltonian
        self.ansatz = ansatz

        # Define some constants
        self.backend = AerSimulator()
        self.num_shots = 10000.0
        self.energies, self.param_vals, self.runtime = list(), list(), list()

    def get_defining_ode(self, measurements: List[dict[str, int]]):
        """
        Construct the dynamics matrix and load vector defining the varQITE
        iteration.
        """
        # Load sampled bitstrings and corresponding frequencies into NumPy arrays
        dtype = np.dtype([("states", int, (self.ansatz.num_qubits,)), ("counts", "f")])
        measurements = [np.fromiter(map(lambda kv: (list(kv[0]), kv[1]), res.items()), dtype) for res in measurements]

        # Set up the dynamics matrix by computing the gradient of each Pauli word
        # with respect to each parameter in the ansatz using the parameter-shift rule
        pauli_terms = [SparsePauliOp(op) for op, _ in self.hamiltonian.label_iter() if set(op) != set("I")]
        Gmat = np.zeros((len(pauli_terms), self.ansatz.num_parameters))
        for i, pauli_word in enumerate(pauli_terms):
            for j, jth_pair in enumerate(zip(measurements[1::2], measurements[2::2])):
                for pm, pm_shift in enumerate(jth_pair):
                    Gmat[i, j] += (-1)**pm * expected_energy(pauli_word, pm_shift)
        
        # Set up the load vector
        curr_energy = expected_energy(self.hamiltonian, measurements[0])
        dvec = np.zeros(len(pauli_terms))
        for i, pauli_word in enumerate(pauli_terms):
            rhs_op_energies = get_ising_energies(pauli_word, measurements[0]["states"])
            rhs_op_energies *= get_ising_energies(self.hamiltonian, measurements[0]["states"]) - curr_energy
            dvec[i] = -np.dot(rhs_op_energies, measurements[0]["counts"]) / self.num_shots
        return Gmat, dvec, curr_energy

    def get_iteration_circuits(self, curr_params: np.array):
        """
        Get the bound circuits that need to be evaluated to step forward
        according to QITE.
        """
        # Use this circuit to estimate your Hamiltonian's expected value
        circuits = [self.ansatz.assign_parameters(curr_params)]

        # Use these circuits to compute gradients
        for k in np.arange(curr_params.shape[0]):
            for j in range(2):
                pm_shift = curr_params.copy()
                pm_shift[k] += (-1)**j * np.pi/2
                circuits += [self.ansatz.assign_parameters(pm_shift)]
        
        # Add measurement gates and return
        [qc.measure_all() for qc in circuits]
        return circuits

    def print_status(self, measurements):
        """
        Print summary statistics describing a QITE run.
        """
        stats = pd.DataFrame({
            "curr_energy": self.energies,
            "num_circuits": [len(measurements)] * len(self.energies),
            "quantum_exec_time": self.runtime 
        })
        stats.index.name = "step"
        print(stats)

    def evolve(self, num_steps: int, lr: float = 0.4):
        """
        Evolve the variational quantum state encoded by ``self.ansatz`` under
        the action of ``self.hamiltonian`` according to varQITE.
        """
        curr_params = np.zeros(self.ansatz.num_parameters)
        for k in range(num_steps):
            # Get circuits and measure on backend
            iter_qc = self.get_iteration_circuits(curr_params)
            job = self.backend.run(iter_qc, shots=self.num_shots)
            q0 = time.time()
            measurements = job.result().get_counts()
            quantum_exec_time = time.time() - q0

            # Update parameters-- set up defining ODE and step forward
            Gmat, dvec, curr_energy = self.get_defining_ode(measurements)
            dcurr_params = np.linalg.lstsq(Gmat, dvec, rcond=1e-2)[0]
            curr_params += lr * dcurr_params

            # Progress checkpoint!
            self.energies.append(curr_energy)
            self.param_vals.append(curr_params.copy())
            self.runtime.append(quantum_exec_time)
            
#####################################################
# Do not modify the code below this line!       #
#####################################################            
            print(k, return_results(self))

def return_results(qit_evolver):
    """
    Return the results of the QITE run.
    """
    return {
        "energies": qit_evolver.energies[-1],
        "runtime": qit_evolver.runtime[-1]
    }

##### Utility functions #####
def compute_cut_size(graph, bitstring):
    """
    Get the cut size of the partition of ``graph`` described by the given
    ``bitstring``.
    """
    cut_sz = 0
    for (u, v) in graph.edges:
        if bitstring[u] != bitstring[v]:
            cut_sz += 1
    return cut_sz

def get_ising_energies(operator: SparsePauliOp, states: np.array):
    """
    Get the energies of the given Ising ``operator`` that correspond to the
    given ``states``.
    """
    # Unroll Hamiltonian data into NumPy arrays
    paulis = np.array([list(ops) for ops, _ in operator.label_iter()]) != "I"
    coeffs = operator.coeffs.real
    
    # Vectorized energies computation
    energies = (-1) ** (states @ paulis.T) @ coeffs
    return energies

def expected_energy(hamiltonian: SparsePauliOp, measurements: np.array):
    """
    Compute the expected energy of the given ``hamiltonian`` with respect to
    the observed ``measurement``.

    The latter is assumed to by a NumPy records array with fields ``states``
    --describing the observed bit-strings as an integer array-- and ``counts``,
    describing the corresponding observed frequency of each state.
    """
    energies = get_ising_energies(hamiltonian, measurements["states"])
    return np.dot(energies, measurements["counts"]) / measurements["counts"].sum()




ansatz = build_ansatz(graph)
ham = build_maxcut_hamiltonian(graph)
qit_evolver = QITEvolver(ham, ansatz)

qit_evolver.evolve(num_steps, lr)

from qiskit_aer import AerSimulator
shots = 100_000

# Sample your optimized quantum state using Aer
backend = AerSimulator()
optimized_state = ansatz.assign_parameters(qit_evolver.param_vals[-1])
optimized_state.measure_all()
counts = backend.run(optimized_state, shots=shots).result().get_counts()

# Find the sampled bitstring with the largest cut value
cut_vals = sorted(((bs, compute_cut_size(graph, bs)) for bs in counts), key=lambda t: t[1])
best_bs = cut_vals[-1][0]

print("\n\n\nbitstring\tcuts\tcounts")
print("----------------------------------")
for i in cut_vals:
    print(i[0],'\t', i[1], '\t', counts[i[0]])
print("\n\n\n")

################## Scoring ##################
G = graph
n = len(G.nodes())
w = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i, j, default=0)
        if temp != 0:
            w[i, j] = 1.0

best_cost_brute = 0

for b in range(2**n):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]

    # Create subgraphs based on the partition
    subgraph0 = G.subgraph([i for i, val in enumerate(x) if val == 0])
    subgraph1 = G.subgraph([i for i, val in enumerate(x) if val == 1])

    bs = "".join(str(i) for i in x)

    # Check if subgraphs are not empty
    if len(subgraph0.nodes) > 0 and len(subgraph1.nodes) > 0:
        cost = 0
        for i in range(n):
            for j in range(n):
                cost = cost + w[i, j] * x[i] * (1 - x[j])
        if best_cost_brute < cost:
            best_cost_brute = cost
            xbest_brute = x
            XS_brut = []
        if best_cost_brute == cost:
            XS_brut.append(bs)

print("Best bitstrings: " + str(XS_brut))  

def final_score(graph, optimal_bitstrings, counts, shots, ansatz):

    num_qubits = graph.number_of_nodes()
    inaccuracy, accuracy_score = 0, 0.0
    
    target_solutions = optimal_bitstrings
    # ensure optimal_bitstrings have the correct length
    optimal_bitstrings_formatted = [bs.zfill(num_qubits) for bs in target_solutions]

    theoretical_amplitude = int(shots/len(optimal_bitstrings_formatted)) 
    for bitstring in optimal_bitstrings_formatted:
        if bitstring in counts:
            inaccuracy += abs(counts[bitstring]-theoretical_amplitude)
        else:
            inaccuracy += theoretical_amplitude
    
    accuracy_score = 1.0 - inaccuracy/shots
    
    # CNOT count penalty
    try:
        transpiled_ansatz = transpile(ansatz, basis_gates=['cx', 'rz', 'sx', 'x'])
        ops_count = transpiled_ansatz.count_ops()
        cx_count = ops_count.get('cx', 0)
    except Exception as e:
        print(f"Error during transpilation or counting ops: {e}")
        cx_count = float('inf')

    num_edges = graph.number_of_edges()
    denominator = (8 * num_edges + cx_count)
    if denominator == 0 :
        efficiency_score = 1.0
    else:
        efficiency_score = (8 * num_edges) / denominator


    score = efficiency_score * accuracy_score

    return np.round(score, 5)

print("Final score: " + str(final_score(graph, XS_brut, counts, shots, ansatz)))




Overwriting submit.py


To run the code - simply import `submit.py` file and the code you wrote above will start executing.

In [5]:
import submit

0 {'energies': -3.9824, 'runtime': 0.0027887821197509766}
1 {'energies': -4.0928, 'runtime': 0.18901300430297852}
2 {'energies': -4.1684, 'runtime': 0.20813417434692383}
3 {'energies': -4.2496, 'runtime': 0.11883807182312012}
4 {'energies': -4.2532, 'runtime': 0.1743788719177246}
5 {'energies': -4.325, 'runtime': 0.19741415977478027}
6 {'energies': -4.4058, 'runtime': 0.14680695533752441}
7 {'energies': -4.5632, 'runtime': 0.1789689064025879}
8 {'energies': -4.6142, 'runtime': 0.1627950668334961}
9 {'energies': -4.713, 'runtime': 0.18346714973449707}
10 {'energies': -4.7986, 'runtime': 0.2022550106048584}
11 {'energies': -4.7104, 'runtime': 0.16295480728149414}
12 {'energies': -4.8024, 'runtime': 0.20106291770935059}
13 {'energies': -4.8834, 'runtime': 0.12371397018432617}
14 {'energies': -4.9614, 'runtime': 0.1919410228729248}
15 {'energies': -5.0446, 'runtime': 0.14729714393615723}
16 {'energies': -5.1212, 'runtime': 0.13978004455566406}
17 {'energies': -5.1788, 'runtime': 0.12634706

If `import submit` was already executed by default python will not re-import the file/module. So, we need to do this manually:

In [6]:
import importlib
importlib.reload(submit)

0 {'energies': -3.9998, 'runtime': 0.2400069236755371}
1 {'energies': -4.1086, 'runtime': 0.12070083618164062}
2 {'energies': -4.1516, 'runtime': 0.13491010665893555}
3 {'energies': -4.2718, 'runtime': 0.13802218437194824}
4 {'energies': -4.3606, 'runtime': 0.14175009727478027}
5 {'energies': -4.3738, 'runtime': 0.1820390224456787}
6 {'energies': -4.4542, 'runtime': 0.12391304969787598}
7 {'energies': -4.5436, 'runtime': 0.12643098831176758}
8 {'energies': -4.3348, 'runtime': 0.11574816703796387}
9 {'energies': -4.4316, 'runtime': 0.15040206909179688}
10 {'energies': -4.4936, 'runtime': 0.12532711029052734}
11 {'energies': -4.5608, 'runtime': 0.13142991065979004}
12 {'energies': -4.6604, 'runtime': 0.20372891426086426}
13 {'energies': -4.76, 'runtime': 0.12800097465515137}
14 {'energies': -4.8224, 'runtime': 0.11325502395629883}
15 {'energies': -4.8812, 'runtime': 0.11364316940307617}
16 {'energies': -4.9562, 'runtime': 0.209456205368042}
17 {'energies': -5.038, 'runtime': 0.1290490627

<module 'submit' from '/Users/karpusenko/Documents/IonQ Projects/56.FLiQ/registration/submit.py'>

## How to submit

The cell below looks at `submit.py` imported code, ispects the source code, and converts it into url safe representation.

In [7]:
import inspect
import base64
import urllib.parse

# Get source code as strings
source_ansatz = inspect.getsource(submit.build_ansatz)
source_ham = inspect.getsource(submit.build_maxcut_hamiltonian)
source_qite = inspect.getsource(submit.QITEvolver)

# Base64-encode the source (as bytes)
encode_ansatz = base64.urlsafe_b64encode(source_ansatz.encode('utf-8')).decode('utf-8')
encode_ham = base64.urlsafe_b64encode(source_ham.encode('utf-8')).decode('utf-8')
encode_qite = base64.urlsafe_b64encode(source_qite.encode('utf-8')).decode('utf-8')


# Now `encode_` can go into a URL
url_ansatz = urllib.parse.quote(encode_ansatz)
url_ham = urllib.parse.quote(encode_ham)
url_qite = urllib.parse.quote(encode_qite)

Source code is sent to our auto-grader server for evaluation. Don't forget to 

In [12]:
graph_number = 2
num_steps = 10
lr = 0.1 #learning rate

import requests
url = f"https://fliq2025.azurewebsites.net/api/run?graph={graph_number}&steps={num_steps}&lr={lr}&team={key}&team_name={team_name}&ansatz={url_ansatz}&ham={url_ham}&qite={url_qite}"
req = requests.post(url)
print(req.text)

Execution completed successfully. Find logs at https://fliq2025.z20.web.core.windows.net/logs/run_log_f2f56e78-05b0-4101-b5f2-88aada89c353.txt


Execution time varies significantly depending on your chosen parameters and code.  You can interrupt the cell execution run at any time; however, it will continue processing on our servers until complete.  Submitting a new run will gracefully end the previous one, which will then report its results.

You can find the current leadership board here: [https://fliq2025.z20.web.core.windows.net/](https://fliq2025.z20.web.core.windows.net/)

NOTE: To run the code on a different graph, simply change the `graph_number` variable and resubmit.  Any currently running job will be automatically stopped, and the leaderboard will be updated with its results up to that point.

If you have any questions don't hesitate to ask them in Discord server.

*Good Luck!!! And Have Fun!!!*